<a href="https://colab.research.google.com/github/suyashmarathe512/suyash/blob/main/Copy_of_RAG_LLM_using_Langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U langchain-community

In [ ]:
!pip install langchain_groq

In [ ]:
!pip install joblib gradio langchain langchain_groq chromadb pypdf

In [ ]:

import os
import joblib
import gradio as gr

from chromadb import Client
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.vectorstores import Chroma
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
# Import ChatGroq from groq_langchain
from langchain_groq import ChatGroq

In [ ]:
db_path = "/content/chroma_db"

def initialize_llm():
    llm = ChatGroq(
        temperature=0,
        groq_api_key="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",
        model_name="llama-3.3-70b-versatile"
    )
    return llm

def create_vector_db():
    """Creates a vector database from PDF files."""
    pdf_dir = "/content/drive/MyDrive/pdf_docs"
    documents = []
    for fn in os.listdir(pdf_dir):
        if fn.endswith(".pdf"):
            loader = PyPDFLoader(os.path.join(pdf_dir, fn))
            documents.extend(loader.load())

    splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    texts = splitter.split_documents(documents)

    embeddings = HuggingFaceBgeEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    vector_db = Chroma.from_documents(
        documents=texts,
        embedding=embeddings,
        persist_directory=db_path
    )
    print("✅ Vector database created and persisted to", db_path)
    return vector_db

def setup_qa_chain(vector_db, llm):
    retriever = vector_db.as_retriever()
    prompt_templates = """

Role: Your name is Brainee, and you are a TherapistBot, an AI trained in Cognitive Behavioral Therapy (CBT) and compassionate listening.

Tone: Maintain a warm, empathetic, and non-judgmental demeanor.

Approach: Utilize evidence-based CBT techniques to help users identify and reframe negative thought patterns.

Engagement: Ask open-ended questions to encourage self-reflection and provide tailored coping strategies.

Limitations: Acknowledge your role as a supportive tool, not a licensed therapist, and encourage users to seek professional help when necessary.

Context:
{context}

User: {question}
Chatbot:
"""
    PROMPT = PromptTemplate(template=prompt_templates, input_variables=['context','question'])
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        chain_type_kwargs={"prompt": PROMPT}
    )
    return qa_chain


In [ ]:
# 1) Build or load your vector DB
# 1) Build or load your vector DB
if not os.path.exists(db_path):
    vector_db = create_vector_db()
else:
    # Initialize HuggingFaceBgeEmbeddings before passing it to Chroma
    embeddings = HuggingFaceBgeEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    vector_db = Chroma(persist_directory=db_path, embedding_function=embeddings)

# 2) Initialize your LLM & QA chain
llm = initialize_llm()
qa_chain = setup_qa_chain(vector_db, llm)


In [ ]:
import gradio as gr

# --- Your existing logic (unchanged) ---
def inference(query: str):
    try:
        answer = qa_chain.run(query=query)
    except Exception as e:
        answer = f"[RAG ERROR] {type(e).__name__}: {e}"
    return answer

def chat_inference(history, query):
    answer = inference(query)
    history = history + [
        {"role": "user", "content": query},
        {"role": "assistant", "content": answer}
    ]
    return history, ""

# --- New frontend layout ---
with gr.Blocks(
    theme=gr.themes.Soft(primary_hue="red"),
    css="""
    /* overall page */
    body, .gradio-container, .gradio-interface {
        background-color: #000 !important;
        color: #FFF !important;
    }

    /* header */
    .db-header { text-align: center; margin-bottom: 1.5rem; }
    .db-title { font-size: 2.5rem; font-weight: bold; margin: .5rem 0; }
    .db-subtitle { font-size: 1rem; color: #FFD1DC; line-height: 1.4; }

    /* chatbot container */
    .gr-chatbot {
        background-color: #000 !important;
        border: 1px solid #66132B !important;
        border-radius: 12px;
    }

    /* message bubbles */
    .gr-chatbot .message {
        background-color: #1A001E !important;
        color: #FFF !important;
    }
    .gr-chatbot .message.user {
        background-color: #3E0E56 !important;
    }
    .gr-chatbot .message.assistant {
        background-color: #2B0A3D !important;
    }

    /* input row centered */
    .input-row {
        display: flex;
        justify-content: center;
        align-items: center;
        margin-top: 1rem;
        gap: 0.5rem;
    }
    .input-row .gr-textbox {
        width: 60% !important;
        background: #3E0E56 !important;
        color: #FFF !important;
        border: 1px solid #66132B !important;
    }
    .gr-button.primary {
        background-color: #FF5E5B !important;
    }
    .gr-button.secondary {
        background-color: #FFA372 !important;
        color: #000 !important;
    }
    """
) as demo:

    # --- Logo + Header ---
    with gr.Column(elem_classes="db-header"):
        gr.Image(
            value="/content/drive/MyDrive/LOGO.jpeg",
            show_label=False,
            height=120
        )
        gr.HTML(
            """
            <div class="db-title">Dr.Brainee</div>
            <div class="db-subtitle">
              Hi, I’m Dr.Brainee — your personal mental health therapy assistant.<br>
              I’m here to listen, support you, and help you through anything on your mind.<br>
              This is a safe, judgment‑free space. Let’s talk. ❤️
            </div>
            """
        )

    # --- Chat Interface (all black) ---
    chatbot = gr.Chatbot(
        label="Conversation",
        type="messages",
        height=450
    )

    # --- Centered input row ---
    with gr.Row(elem_classes="input-row"):
        txt = gr.Textbox(
            show_label=False,
            placeholder="How are you feeling today?",
            lines=2
        )
        send_btn = gr.Button("Send", variant="primary")
        clear_btn = gr.Button("Clear", variant="secondary")

    # --- Events ---
    send_btn.click(fn=chat_inference, inputs=[chatbot, txt], outputs=[chatbot, txt])
    txt.submit(fn=chat_inference, inputs=[chatbot, txt], outputs=[chatbot, txt])
    clear_btn.click(lambda: ([], ""), [], [chatbot, txt])

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://250487ef470bcbe279.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
